# Khai báo thư viện

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import datetime, time, json
from string import punctuation
from keras.layers import Input
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,GlobalMaxPooling1D,LeakyReLU,Dense, Dropout,Conv1D, SpatialDropout1D,Reshape,Concatenate, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from collections import defaultdict
from keras.layers.merge import concatenate
from keras.models import Model
import time
import sklearn
import random 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import ntpath
import keras


Using TensorFlow backend.


# Load data

In [2]:
# x3 data clean
from pathlib import Path
input_dir = Path('train/OutText')
data_files = input_dir.glob('**/*.asm')
data_total = []
label_total = []
for file in data_files:
    with open(file, 'r') as f:
        data = f.read().replace('\x00','').lower()
        data_total.append(data)
        if 'clean' in str(file):
            label_total.append([1,0])
        else:
            label_total.append([0,1])

## sontdc_loaddata

In [2]:
# x3 data clean
from pathlib import Path
data_total = []
label_total = []
# clean_js path
input_clean_dir = Path('H:/DoAnTotNghiep/DAMH/data_crawl/sontdc_data_js/clean_js/res')
data_clean_files = input_clean_dir.glob('*')
for file in data_clean_files:
    with open(file, 'r') as f:
        data = f.read().replace('\x00', '').lower()
        data_total.append(data)
        data_total.append(data)
        label_total.append([1, 0])
        label_total.append([1, 0])
# malware_js path
input_malware_dir = Path('H:/DoAnTotNghiep/DAMH/data_crawl/sontdc_data_js/virus_js/res')
data_malware_files = input_malware_dir.glob('*')
for file in data_malware_files:
    with open(file, 'r') as f:
        data = f.read().replace('\x00', '').lower()
        data_total.append(data)
        label_total.append([0, 1])


In [3]:
len(label_total)

5222

In [4]:
len(data_total)

5222

# xử lý data

In [5]:
RANDOM_STATE = 42

# Split train & test
text_train, text_test, y_train, y_test = train_test_split(data_total, label_total, test_size=0.1, random_state=RANDOM_STATE)

#num_words = 10000
# Tokenize and transform to integer index
tokenizer = Tokenizer(oov_token='UNK',filters='\t\n')
tokenizer.fit_on_texts(text_train)

#change to unk it word > num_words
# mặc định thứ tự các từ đã sắp xếp theo tuần suất nên có thể loại các từ vị trí cao
#tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= num_words}
#tokenizer.word_index[tokenizer.oov_token] = num_words + 1

X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = max(len(x) for x in X_train) # longest text in train set

# Add pading to ensure all vectors have same dimensionality
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

MemoryError: Unable to allocate 6.98 GiB for an array with shape (4699, 398719) and data type int32

In [6]:
import pickle
# loading
with open('tokenizer_10000_1902.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: 'tokenizer_10000_1902.pickle'

In [6]:
RANDOM_STATE = 42

# Split train & test
text_train, text_test, y_train, y_test = train_test_split(data_total, label_total, test_size=0.1, random_state=RANDOM_STATE)


In [7]:
tokenizer = Tokenizer(oov_token='UNK',filters='\t\n')
tokenizer.fit_on_texts(text_train)

In [26]:
X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

In [6]:
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [7]:
vocab_size

5153

In [8]:
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

In [9]:
MaxSize = 20000

In [10]:
for index2 in range(len(X_train)):
    if len(X_train[index2]) > MaxSize:
        del X_train[index2][(MaxSize):] 
for index3 in range(len(X_test)):
    if len(X_test[index3]) > MaxSize:
        del X_test[index3][(MaxSize):]         

In [32]:
X_train = pad_sequences(X_train, padding='post', maxlen=MaxSize)
X_test = pad_sequences(X_test, padding='post', maxlen=MaxSize)

In [33]:
X_train

array([[ 11,  15,   8, ...,   0,   0,   0],
       [ 25,   6, 666, ...,   0,   0,   0],
       [ 11,  15,   8, ...,   0,   0,   0],
       ...,
       [ 11,  15,   8, ...,   0,   0,   0],
       [ 16,  34,   5, ...,   0,   0,   0],
       [ 33,   6,   5, ...,   0,   0,   0]])

In [34]:
vocab_size

5153

In [35]:
X_train[0]

array([11, 15,  8, ...,  0,  0,  0])

# Thiết kế mạng

In [36]:
# Define CNN architecture

embedding_dim = 100

model = Sequential()
model.add(keras.layers.Embedding(vocab_size + 1, embedding_dim, input_length=MaxSize))
model.add(keras.layers.Conv1D(128, 5, activation='relu'))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(10, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(2, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20000, 100)        515400    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 19996, 128)        64128     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 22        
Total params: 580,840
Trainable params: 580,840
Non-trainable params: 0
________________________________________________

# Chạy huấn luyện

In [46]:
# Fit model
history = model.fit(X_train, y_train,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)


Train on 4699 samples, validate on 523 samples
Epoch 1/5
4699/4699 [==============================] - 567s 121ms/step - loss: 0.0390 - accuracy: 0.9836 - val_loss: 0.1301 - val_accuracy: 0.9751
Epoch 2/5
4699/4699 [==============================] - 562s 120ms/step - loss: 0.0330 - accuracy: 0.9874 - val_loss: 0.1571 - val_accuracy: 0.9771
Epoch 3/5
4699/4699 [==============================] - 564s 120ms/step - loss: 0.0368 - accuracy: 0.9851 - val_loss: 0.1628 - val_accuracy: 0.9732
Epoch 4/5
4699/4699 [==============================] - 562s 120ms/step - loss: 0.0402 - accuracy: 0.9834 - val_loss: 0.1738 - val_accuracy: 0.9751
Epoch 5/5
4699/4699 [==============================] - 561s 119ms/step - loss: 0.0442 - accuracy: 0.9840 - val_loss: 0.1441 - val_accuracy: 0.9771


In [8]:
model.save('JS_CNN_0805_15Epoc.md')

NameError: name 'model' is not defined

In [40]:
from pathlib import Path
input_dir = Path('test/OutText/virus')
data_files = input_dir.glob('**/*.asm')
data_total = []
label_total = []
a=0
b=0
i=0
for file in data_files:
    with open(file, 'r') as f:
        data = f.read().replace('\x00','').lower()
        data_total.append(data)
        X_sample = tokenizer.texts_to_sequences(data_total)
        X_sample = pad_sequences(X_sample, padding='post', maxlen=200000)
        y_sample = model_load.predict(X_sample)
        if y_sample[0][0]>y_sample[0][1]:
            a+=1
        else:
            b+=1
        data_total = []

In [41]:
a

5

In [42]:
b

981

## Bắt đầu test từ đây

In [25]:
from pathlib import Path
input_dir = Path("H:/DoAnTotNghiep/DAMH/data_crawl/tap_test_virus/res")
data_files = input_dir.glob('*')
data_total = []
label_total = []

# them vao mang luu ten de tim mau nhan nham
file_names = []

a=0
b=0
i=0
for file in data_files:
    file_names.append(file)
    with open(file, 'r') as f:
        data = f.read().replace('\x00','').lower()
        data_total.append(data)
    i+=1

In [26]:
len(data_total)

2122

In [11]:
from keras.models import load_model
model_load = load_model('JS_CNN_0805_15Epoc.md')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [12]:
X_sample = tokenizer.texts_to_sequences(data_total)
X_sample = pad_sequences(X_sample, padding='post', maxlen=20000)





In [23]:
X_sample

array([[ 20,   6, 628, ...,   0,   0,   0],
       [ 11,  15,   8, ...,   0,   0,   0],
       [ 25,   6, 659, ...,   0,   0,   0],
       ...,
       [ 20,   8,   3, ...,   0,   0,   0],
       [ 15,   8,  22, ...,   0,   0,   0],
       [ 15,   8,   4, ...,   0,   0,   0]])

In [14]:
y_sample = model_load.predict(X_sample)

In [20]:
y_sample


array([[6.2584877e-07, 9.9999952e-01],
       [2.7030706e-05, 9.9997580e-01],
       [0.0000000e+00, 1.0000000e+00],
       ...,
       [2.3841858e-07, 9.9999976e-01],
       [0.0000000e+00, 1.0000000e+00],
       [1.1404058e-04, 9.9989498e-01]], dtype=float32)

In [27]:
y_sample[0][0]

0.999995

In [27]:
a=0
b=0
for x in range(len(y_sample)):
    if y_sample[x][0]>y_sample[x][1]:
        a+=1
        print(x, file_names[x])
    else:
        b+=1
        

5 H:\DoAnTotNghiep\DAMH\data_crawl\tap_test_virus\res\static_0031f0c0e3a4c4a44ca7db79c1cd0587c3ab4cf1e16402613399673f0e967090
14 H:\DoAnTotNghiep\DAMH\data_crawl\tap_test_virus\res\static_00766f24eb8521bcbf4357c77b5dcaf4a43c5346d98b4b4fea49f13cf2d88087
20 H:\DoAnTotNghiep\DAMH\data_crawl\tap_test_virus\res\static_00a2f1e4f098d807a29de0a9d825dd951da3d02bec9f873523be99bdd4a4e574
36 H:\DoAnTotNghiep\DAMH\data_crawl\tap_test_virus\res\static_014284e544415350fda4ece23ee57e25a2728cadff487f31107474d6a5fe8e9b
41 H:\DoAnTotNghiep\DAMH\data_crawl\tap_test_virus\res\static_017ba30cb64aa42b52e267a9f7c9e4ce0e3f74c2fe8268481c5432742e4c3419
50 H:\DoAnTotNghiep\DAMH\data_crawl\tap_test_virus\res\static_01dc50cd3274905663fad0ebad59490f0701586a7505c76cdd71b7a9272bea32
63 H:\DoAnTotNghiep\DAMH\data_crawl\tap_test_virus\res\static_0268197d39e1f00dc5ba2e20cf7f9f620f021194b49e5c1b38a2cba50863a56e
68 H:\DoAnTotNghiep\DAMH\data_crawl\tap_test_virus\res\static_02a862a61dd0238b07c06e13ac06f4e13e3d91b35d59d8ad9a

In [22]:
a

128

## Check mẫu nhận nhầm

In [29]:
from pathlib import Path
input_dir = Path('C:\\Users\\sontdc\\Desktop\\virus_nham')
data_files = input_dir.glob('*')
data_total = []
label_total = []

# them vao mang luu ten de tim mau nhan nham
file_names = []

a=0
b=0
i=0
for file in data_files:
    file_names.append(file)
    with open(file, 'r') as f:
        data = f.read().replace('\x00','').lower()
        data_total.append(data)
    i+=1

In [30]:
X_sample = tokenizer.texts_to_sequences(data_total)
X_sample = pad_sequences(X_sample, padding='post', maxlen=20000)

In [31]:
y_sample = model_load.predict(X_sample)

In [32]:
a=0
b=0
for x in range(len(y_sample)):
    if y_sample[x][0]>y_sample[x][1]:
        a+=1
#         print(x, file_names[x])
    else:
        b+=1

In [33]:
a

6

# Đồ thị hóa quá trình huấn luyện

In [22]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

%matplotlib inline
plot_history(history)

NameError: name 'history' is not defined

# Dự đoán thử

In [12]:
%%time

X_sample = ['Đăng ký mua nhà ở VinHome']
X_sample = tokenizer.texts_to_sequences(X_sample)
X_sample = pad_sequences(X_sample, padding='post', maxlen=maxlen)

y_sample = model.predict_classes(X_sample).flatten().tolist()

print('Prediction: ',y_sample)

Prediction:  [1]
CPU times: user 321 ms, sys: 54.1 ms, total: 376 ms
Wall time: 162 ms


In [14]:
%%time

X_sample = ['chào Bkav']
X_sample = tokenizer.texts_to_sequences(X_sample)
X_sample = pad_sequences(X_sample, padding='post', maxlen=maxlen)

y_sample = model.predict_classes(X_sample).flatten().tolist()

print('Prediction: ',y_sample)

Prediction:  [0]
CPU times: user 321 ms, sys: 59.2 ms, total: 380 ms
Wall time: 156 ms


# lưu và load model

In [10]:
from keras.models import load_model

model.save('antispam_team2_2.md')
model_load = load_model('antispam_team2_2.md')

# Lưu và load Tokenizer (thông tin bộ từ)

In [11]:
import pickle

# saving
with open('tokenizer_2.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer_2.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)